In [ ]:
%pylab inline
import os
import shutil
from glob import glob
import pysumma as ps
import xarray as xr
import pandas as pd
from pathlib import Path

In [ ]:
veg_igbp = pd.read_csv('./VEGPARM_IGBP_MODIS_NOAH.TBL', 
                       index_col=-1, skipinitialspace=True)
veg_igbp.index = veg_igbp.index.map(lambda x: x.strip().replace("'", ""))

soil_rosetta = pd.read_csv('./SOILPARM_ROSETTA.TBL', 
                           index_col=-1, skipinitialspace=True)
soil_rosetta.index = soil_rosetta.index.map(lambda x: x.strip().replace("'", ""))

In [ ]:
sites = os.listdir('../sites/')


bad_sites = []
sim_sites = [s for s in sites if s not in bad_sites]

seed = 50334
np.random.seed(seed)
np.random.shuffle(sim_sites)
len(sim_sites)

nfold = 5
equal_divisor_len = nfold * (len(sim_sites) // nfold)
kfold_test_sites = list(np.array(sim_sites[0:equal_divisor_len]).reshape(nfold, -1))
kfold_test_sites[-1] = np.hstack([kfold_test_sites[-1], sim_sites[equal_divisor_len:]])
kfold_train_sites = [list(set(sim_sites) - set(test_sites)) for test_sites in kfold_test_sites]

model_src = [f'../models/train_states_qle_set_{n}.txt' for n in range(nfold)]
model_rel_dest = [f'../params/train_states.txt' for n in range(nfold)]
executable = '../state_ml_summa/bin/ml_summa'

kfold_configs = []
for n in range(nfold) :
    kfold_sites = kfold_test_sites[n]
    config = {site: {'file_manager': f'../sites/{site}/file_manager.txt'} for site in kfold_sites}
    
    for s, c in config.items():
        # set model in file manager
        fman = ps.FileManager(c['file_manager'], name='')
        fman.options.append(ps.file_manager.FileManagerOption('neuralNetFile', model_rel_dest[n]))
        fman['outFilePrefix'] = f'state_nn_output'
        
        # Create a new initial conditions
        #ic_src =  f'../sites/{s}/params/initial_conditions.nc'
        #ic_dest = f'../sites/{s}/params/state_initial_conditions.nc'
        #shutil.copy(ic_src, ic_dest)
        fman['initConditionFile'] = f'../params/state_initial_conditions.nc'
        
        # save file manager
        fman.write(path=str(fman.original_path).replace('file_manager', f'file_manager_state_NN'))
        
        # copy neural net file over to params
        nn_dest = c['file_manager'].replace('file_manager.txt', model_rel_dest[n].replace('../', ''))
        shutil.copy(model_src[n], nn_dest)
        c['file_manager'] = f'../sites/{s}/file_manager_state_NN.txt'
    kfold_configs.append(config)

config = kfold_configs[0]
for kfc in kfold_configs[1:]:
    config.update(kfc)

In [ ]:
ens = ps.Ensemble(executable, config, num_workers=21)
ens.run('local')
summary = ens.summary()

In [ ]:
failed = summary['error']
success = summary['success']

len(failed), len(success)

In [ ]:
failed

In [ ]:
print(ens.simulations[summary['error'][0]].stdout[-500:])
print(ens.simulations[summary['error'][0]].stdout[0:500])